The aim of the transformation pipeline is to prepare the data in a way that it can be easily consumed by your downstream processes, whether that's a machine learning model, a recommendation system, or any other analytics

1. Convert the date_of_job_post column into a datetime format.
2. Extract the month, day, and year from the date_of_job_post and save them as separate columns.
3. Convert categorical columns (title, job_location, company_name, and job_type) to a numerical format, probably using techniques like one-hot encoding or label encoding, depending on the nature of the data and the modeling techniques you plan to use.


